In [1]:
# URL Used: https://covidtracking.com/data

# Import Libraries
import pandas as pd


In [2]:
# # First dataframe checked
# csv_file = "OG_data_files/countrywide_daily_testing.csv"
# testing_df = pd.read_csv(csv_file)
# testing_df.head(50)
# sorted_df = testing_df.sort_values("Date")
# sorted_df.tail(10)

In [3]:
# Pull in CSV file using Pandas DF
testing_csv = "Resources/all_states_testing.csv"
by_state_df = pd.read_csv(testing_csv)

# Check Column header names to craft new DF
by_state_df.columns

Index(['date', 'state', 'dataQualityGrade', 'death', 'deathConfirmed',
       'deathIncrease', 'deathProbable', 'hospitalized',
       'hospitalizedCumulative', 'hospitalizedCurrently',
       'hospitalizedIncrease', 'inIcuCumulative', 'inIcuCurrently', 'negative',
       'negativeIncrease', 'negativeTestsAntibody',
       'negativeTestsPeopleAntibody', 'negativeTestsViral',
       'onVentilatorCumulative', 'onVentilatorCurrently', 'pending',
       'positive', 'positiveCasesViral', 'positiveIncrease', 'positiveScore',
       'positiveTestsAntibody', 'positiveTestsAntigen',
       'positiveTestsPeopleAntibody', 'positiveTestsPeopleAntigen',
       'positiveTestsViral', 'recovered', 'totalTestEncountersViral',
       'totalTestEncountersViralIncrease', 'totalTestResults',
       'totalTestResultsIncrease', 'totalTestsAntibody', 'totalTestsAntigen',
       'totalTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'totalTestsPeopleViral', 'totalTestsPeopleViralIncrease',
       'total

In [4]:
# Craft and sort new DF to use for database
new_state_df = pd.DataFrame(by_state_df[['date', 'state','negative','negativeIncrease', 'positive','positiveIncrease', 'death','deathProbable','pending','totalTestResults']])

# Remove irrelevant states (districts)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='AS'].index, inplace=True)
# new_state_df.drop(new_state_df.loc[new_state_df['state']=='DC'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='GU'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='MP'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='PR'].index, inplace=True)
new_state_df.drop(new_state_df.loc[new_state_df['state']=='VI'].index, inplace=True)



# Sort the DF by 'state' then 'date'
sorted_state_df = new_state_df.sort_values(['state', 'date'])
sorted_state_df.head()
sorted_state_df


,date,state,negative,negativeIncrease,positive,positiveIncrease,death,deathProbable,pending,totalTestResults
10731,20200306,AK,8.0,0,0.0,0,0.0,NaN,1.0,8.0
10697,20200307,AK,12.0,4,0.0,0,0.0,NaN,2.0,12.0
10645,20200308,AK,14.0,2,0.0,0,0.0,NaN,6.0,14.0
10592,20200309,AK,23.0,9,0.0,0,0.0,NaN,9.0,23.0
10544,20200310,AK,23.0,0,0.0,0,0.0,NaN,9.0,23.0
...,...,...,...,...,...,...,...,...,...,...
278,20200910,WY,81100.0,416,4199.0,48,42.0,NaN,NaN,85299.0
168,20200911,WY,82091.0,991,4264.0,65,42.0,NaN,NaN,86355.0
165,20200912,WY,82091.0,0,4297.0,33,42.0,NaN,NaN,86388.0
56,20200913,WY,82091.0,0,4346.0,49,42.0,NaN,NaN,86437.0


In [5]:
# Check if we deleted the correct number and state names
states = sorted_state_df['state']
for state in states:
#     print(state)
    if sorted_state_df['state'].nunique() == 51:
        print('Yes! We have 50 states in the df')
        break

Yes! We have 50 states in the df


In [6]:
# Import CSV with states Name and Code to match up with other DF
states_csv = "Resources/states_abbrev.csv"
states_df = pd.read_csv(states_csv)
states_code_df = pd.DataFrame(states_df[['State','Code']])

# Rename column headers to merge DF's
states_code_df = states_code_df.rename(columns={'State':'state_name', 'Code':'state'})

# # Drop District of Columbia code
# # df[df.name != 'Tina']
# states_code_df = states_code_df.drop(states_code_df.index[8])

# # sort the df
# states_code_df = states_code_df.sort_values(by='state_name').reset_index(drop=True)

# # Turn back into a DF
# states_code_df = pd.DataFrame(states_code_df)

states_code_df.head()

,state_name,state
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [7]:
# Check number of states is correct
states = states_code_df['state']
for state in states:
#     print(state)
    if states_code_df['state'].nunique() == 51:
        print('Yes! We have 50 states in the df')
        break

Yes! We have 50 states in the df


In [8]:
# Merge sorted_state_df and states_code_df in order to match sorted_state_df and @Authum's DF created
states_testing_df = pd.merge(sorted_state_df, states_code_df, on='state', how='outer')
states_testing_df = states_testing_df[['date', 'state','state_name','negative','negativeIncrease', 'positive','positiveIncrease', 'death','deathProbable','pending','totalTestResults']]

# Double check correct number of states and state name
print(states_testing_df['state_name'].nunique())
print(states_testing_df['state'].nunique())

# Verify DC is still in dataset
states_testing_df.loc[states_testing_df['state']=='DC']

#Show df one last time...
states_testing_df = states_testing_df.astype({'date': 'str'})
states_testing_df.dtypes

51
51


date                 object
state                object
state_name           object
negative            float64
negativeIncrease      int64
positive            float64
positiveIncrease      int64
death               float64
deathProbable       float64
pending             float64
totalTestResults    float64
dtype: object

In [9]:
# Date to datetime
states_testing_df['date'] = pd.to_datetime(states_testing_df['date'], format = "%Y%m%d").dt.strftime('%Y-%m-%d') 
states_testing_df

,date,state,state_name,negative,negativeIncrease,positive,positiveIncrease,death,deathProbable,pending,totalTestResults
0,2020-03-06,AK,Alaska,8.0,0,0.0,0,0.0,NaN,1.0,8.0
1,2020-03-07,AK,Alaska,12.0,4,0.0,0,0.0,NaN,2.0,12.0
2,2020-03-08,AK,Alaska,14.0,2,0.0,0,0.0,NaN,6.0,14.0
3,2020-03-09,AK,Alaska,23.0,9,0.0,0,0.0,NaN,9.0,23.0
4,2020-03-10,AK,Alaska,23.0,0,0.0,0,0.0,NaN,9.0,23.0
...,...,...,...,...,...,...,...,...,...,...,...
9986,2020-09-10,WY,Wyoming,81100.0,416,4199.0,48,42.0,NaN,NaN,85299.0
9987,2020-09-11,WY,Wyoming,82091.0,991,4264.0,65,42.0,NaN,NaN,86355.0
9988,2020-09-12,WY,Wyoming,82091.0,0,4297.0,33,42.0,NaN,NaN,86388.0
9989,2020-09-13,WY,Wyoming,82091.0,0,4346.0,49,42.0,NaN,NaN,86437.0


In [10]:
# Export crafted df to new CSV file
states_testing_df.to_csv("CSV_files_created/states_testing.csv", index=False)
